In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from utils.train.build import TrainSix
from utils.plotter import plot

In [3]:
import awkward as ak
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import vector
vector.register_awkward()

In [4]:
def p4(pt, eta, phi, m):
    return vector.obj(pt=pt, eta=eta, phi=phi, m=m)

In [5]:
filename = '../inputs/NMSSM_XYH_YToHH_6b_MX_700_MY_400_training_set.root'

In [6]:
training = TrainSix(filename)

Initializing tree.
Identifying signal.


In [7]:
signal = training.correct_mask
cbkgd  = training.incorrect_mask

In [8]:
jet_pt = training.get_t6('jet_pt')
jet_eta = training.get_t6('jet_eta')
jet_phi = training.get_t6('jet_phi')
jet_m = training.get_t6('jet_m')
jet_btag = training.get_t6('jet_btag')

In [10]:
sig_btag = jet_btag[signal].to_numpy()
bkg_btag = jet_btag[cbkgd].to_numpy()

In [11]:
sig_pt  = ak.unzip(ak.combinations(jet_pt[signal], 6))
sig_eta = ak.unzip(ak.combinations(jet_eta[signal], 6))
sig_phi = ak.unzip(ak.combinations(jet_phi[signal], 6))
sig_m   = ak.unzip(ak.combinations(jet_m[signal], 6))

In [12]:
sig_pt = [ak.flatten(pt) for pt in sig_pt]
sig_eta = [ak.flatten(pt) for pt in sig_eta]
sig_phi = [ak.flatten(pt) for pt in sig_phi]
sig_m = [ak.flatten(pt) for pt in sig_m]

In [13]:
sig_p4 = [p4(pt,eta,phi,m) for pt,eta,phi,m in zip(sig_pt,sig_eta,sig_phi,sig_m)]
signal_input = [[p4.E, p4.px, p4.py, p4.pz] for p4 in sig_p4]
signal_input = np.asarray(signal_input)
signal_input.shape

(6, 4, 921985)

In [14]:
bkg_pt  = ak.unzip(ak.combinations(jet_pt[cbkgd], 6))
bkg_eta = ak.unzip(ak.combinations(jet_eta[cbkgd], 6))
bkg_phi = ak.unzip(ak.combinations(jet_phi[cbkgd], 6))
bkg_m   = ak.unzip(ak.combinations(jet_m[cbkgd], 6))

In [15]:
bkg_pt = [ak.flatten(pt) for pt in bkg_pt]
bkg_eta = [ak.flatten(pt) for pt in bkg_eta]
bkg_phi = [ak.flatten(pt) for pt in bkg_phi]
bkg_m = [ak.flatten(pt) for pt in bkg_m]

In [16]:
bkg_p4 = [p4(pt,eta,phi,m) for pt,eta,phi,m in zip(bkg_pt,bkg_eta,bkg_phi,bkg_m)]
bkgd_input = [[p4.E, p4.px, p4.py, p4.pz] for p4 in bkg_p4]
bkgd_input = np.asarray(bkgd_input)
bkgd_input.shape

(6, 4, 921983)

In [17]:
from lbn import LBN

In [56]:
lbn = LBN(11, 5, boost_mode=LBN.PRODUCT)

In [57]:
bkgd_lbn_features = lbn(np.transpose(bkgd_input, (2,0,1))[:1000,...], features=["E","px","py","pz"])

In [58]:
bkgd_lbn_features

<tf.Tensor: shape=(1000, 220), dtype=float32, numpy=
array([[119.88961   , 115.38278   , 156.85626   , ...,  19.282246  ,
          4.5712204 ,  39.661877  ],
       [ 39.55864   ,  39.140457  ,  41.646034  , ...,  -7.8888397 ,
          4.5588875 ,  -9.153503  ],
       [ 39.210964  ,  36.999928  ,  39.933918  , ..., -35.35868   ,
         -1.0717802 , -40.943146  ],
       ...,
       [ 65.19505   ,  55.99904   ,  66.11459   , ..., -20.790644  ,
         67.103004  ,  -5.5584183 ],
       [ 38.769573  ,  42.761513  ,  49.96231   , ...,  60.610588  ,
         24.862621  ,  42.332363  ],
       [ 42.268147  ,  42.954987  ,  43.73986   , ...,   0.16486359,
         24.551682  , -19.297829  ]], dtype=float32)>

In [59]:
signal_lbn_features = lbn(np.transpose(signal_input, (2,0,1))[:1000,...], features=["E","px","py","pz"])

In [60]:
signal_lbn_features

<tf.Tensor: shape=(1000, 220), dtype=float32, numpy=
array([[ 41.37749  ,  40.849068 ,  45.23269  , ...,   6.114486 ,
        -11.578175 ,  21.668858 ],
       [ 40.56363  ,  40.671906 ,  46.083717 , ...,  25.53328  ,
         51.739098 ,  35.730377 ],
       [ 34.721    ,  34.37339  ,  36.45673  , ...,   0.9926491,
         -4.4729347,  -3.9192867],
       ...,
       [ 28.150589 ,  30.370838 ,  32.798862 , ..., -20.485039 ,
         -7.7699356, -14.107376 ],
       [ 33.147663 ,  33.85107  ,  36.877136 , ..., -26.621246 ,
         -5.852421 ,  -8.718517 ],
       [ 36.608673 ,  37.51292  ,  39.341225 , ..., -29.519386 ,
         -5.000145 , -23.983055 ]], dtype=float32)>

In [44]:
fig, axs = plt.subplots(nrows=11, ncols=4, figsize=(10,20))
for i,ax in enumerate(axs.flatten()):
    bkgd_mean = np.mean(bkgd_lbn_features[:,i].numpy())
    bkgd_std = np.std(bkgd_lbn_features[:,i].numpy())
    baxrange = [bkgd_mean - 3*bkgd_std, bkgd_mean + 3*bkgd_std]
    
    signal_mean = np.mean(signal_lbn_features[:,i].numpy())
    signal_std = np.std(signal_lbn_features[:,i].numpy())
    saxrange = [signal_mean - 3*signal_std, signal_mean + 3*signal_std]
    
    axrange = [baxrange[0] if baxrange[0] < saxrange[0] else saxrange[0], baxrange[1] if baxrange[1] > saxrange[1] else saxrange[1]]
    
    bins = np.linspace(axrange[0], axrange[1], 50)
    _ = plot(x=signal_lbn_features[:,i].numpy(), bins=bins, fig=fig, ax=ax)
    _ = plot(x=bkgd_lbn_features[:,i].numpy(), bins=bins, fig=fig, ax=ax)
    
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
import tensorflow as tf

In [26]:
signal_lbn_features = tf.concat([signal_lbn_features, sig_btag], axis=1)

In [27]:
bkgd_lbn_features = tf.concat([bkgd_lbn_features, bkg_btag], axis=1)

In [28]:
features = tf.concat([signal_lbn_features, bkgd_lbn_features], axis=0)

In [ ]:
plt.hist2d()